In [2]:
# !pip install torch transformers datasets scikit-learn

In [1]:
import os

os.chdir('/content')
os.getcwd()

'/content'

In [2]:
os.listdir()

['.config', 'Train_data.csv', 'sample_data']

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv("/content/Train_data.csv")

# Encode labels
label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df["label"])

# Train-test split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["text"], df["label_id"], test_size=0.30, random_state=42, stratify=df["label_id"]
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.50, random_state=42, stratify=temp_labels
)


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=64)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Step 4: Create PyTorch Dataset

In [5]:
import torch

class SymptomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {"labels": torch.tensor(self.labels[idx])}

train_dataset = SymptomDataset(train_encodings, train_labels.tolist())
val_dataset = SymptomDataset(val_encodings, val_labels.tolist())
test_dataset = SymptomDataset(test_encodings, test_labels.tolist())


Step 5: Model Definition (BERT + Classification Head)

In [6]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=24
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# !pip install --upgrade transformers

Step 6: Training Setup

In [ ]:
# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     load_best_model_at_end=True
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer
# )

from transformers import Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification
import torch

# Load model & tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=24)

# Define TrainingArguments (no evaluation_strategy here)
training_args = TrainingArguments(
    output_dir="./results",
    save_steps=1000,          # save checkpoint every N steps
    save_total_limit=1,       # keep only last checkpoint
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=50,
    do_eval=True              # make sure evaluation is enabled
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # from your earlier code
    eval_dataset=val_dataset,     # from your earlier code
    tokenizer=tokenizer
)

# Train with manual evaluation each epoch
for epoch in range(int(training_args.num_train_epochs)):
    print(f"\n===== Epoch {epoch+1} / {training_args.num_train_epochs} =====")
    trainer.train()
    print("\n*** Running Evaluation ***")
    trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2758361453.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



===== Epoch 1 / 5 =====


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bharath-ts91 (bharath-ts91-paytm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss


Step 7: Train the Model

In [ ]:

# trainer.train()

Step 8: Evaluation

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=1)

from sklearn.metrics import classification_report

print(classification_report(test_labels, pred_labels, target_names=label_encoder.classes_))


Step 9: Deployment-Ready Inference Function

In [ ]:
def predict_disease(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# Example
print(predict_disease("high fever, severe headache, joint pain"))
